In [19]:
import pandas as pd 
import talib

data_path = '../input/'
train = pd.read_pickle(data_path+'/train.pkl')
train.head()

,timestamp,Asset_ID,Count,Open,High,Low,Close,Volume,VWAP,Target
0,1514764860,2,40.0,2376.5800,2399.5000,2357.1400,2374.5900,19.233005,2373.116392,-0.004218
1,1514764860,0,5.0,8.5300,8.5300,8.5300,8.5300,78.380000,8.530000,-0.014399
2,1514764860,1,229.0,13835.1940,14013.8000,13666.1100,13850.1760,31.550062,13827.062093,-0.014643
3,1514764860,5,32.0,7.6596,7.6596,7.6567,7.6576,6626.713370,7.657713,-0.013922
4,1514764860,7,5.0,25.9200,25.9200,25.8740,25.8770,121.087310,25.891363,-0.008264


In [20]:
import numpy as np

def moving_average(x, window):
    b = np.ones(window) / window

    return np.convolve(x, b, mode='same')

In [21]:
asset_group_close = train.groupby('Asset_ID').Close

In [22]:

train['moving_average'] = asset_group_close.transform(lambda x: x.rolling(window=15).mean())
train.moving_average

0                   NaN
1                   NaN
2                   NaN
3                   NaN
4                   NaN
               ...     
24236801     156.986317
24236802    2428.804866
24236803       0.091287
24236804       0.281847
24236805     232.334100
Name: moving_average, Length: 24236806, dtype: float64

In [23]:
train['moving_average'] = asset_group_close.transform(lambda x: moving_average(x.values, 15))
train.moving_average

0           1264.049111
1              4.510827
2           7338.796400
3              4.062320
4             14.022100
               ...     
24236801      83.758457
24236802    1296.253827
24236803       0.048678
24236804       0.150373
24236805     123.791700
Name: moving_average, Length: 24236806, dtype: float64

In [24]:
def moving_average(a, n):
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret / n


In [25]:
train['moving_average'] = asset_group_close.transform(lambda x: moving_average(x.values, 15))
train.moving_average

0            158.306000
1              0.568667
2            923.345067
3              0.510507
4              1.725133
               ...     
24236801     156.986317
24236802    2428.804866
24236803       0.091287
24236804       0.281847
24236805     232.334100
Name: moving_average, Length: 24236806, dtype: float64

In [26]:
import talib

In [27]:
train['ht_trendline'] = asset_group_close.transform(lambda x: talib.HT_TRENDLINE(x))

In [28]:
train['ht_trendline'] = asset_group_close.transform(lambda x: talib.HT_TRENDLINE(x.values))

In [31]:
for id in train.Asset_ID.unique():
# ADX - Average Directional Movement Index (平均方向性指数)
# トレンドの存在を確認するための指標
    x = train[train.Asset_ID==id]
    train.loc[train.Asset_ID==id, 'adx'] = talib.ADX(x.High.values, x.Low.values, x.Close.values, timeperiod=14)


In [33]:
for id in train.Asset_ID.unique():
# ADX - Average Directional Movement Index (平均方向性指数)
# トレンドの存在を確認するための指標
    x = train[train.Asset_ID==id]
    train.loc[train.Asset_ID==id, 'adx'] = talib.ADX(x.High, x.Low, x.Close, timeperiod=14)
train.adx

0                 NaN
1                 NaN
2                 NaN
3                 NaN
4                 NaN
              ...    
24236801    18.514184
24236802    18.650841
24236803    23.033844
24236804    15.025228
24236805    16.096834
Name: adx, Length: 24236806, dtype: float64